# How “good” is your model, and how can you make it better?

What distinguishes “true artists” from “one-hit wonders” in machine learning is an understanding of how a model performs with respect to different data. This hands-on tutorial will show you how to use scikit-learn’s model evaluation functions to evaluate different models in terms of accuracy and generalisability, and search for optimal parameter configurations.

## Load the required libraries

In [ ]:
import scipy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.cross_validation as cv

from sklearn import preprocessing
from sklearn import metrics

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.grid_search import GridSearchCV

%matplotlib inline

## Exploring and pre-processing data

The first thing you will need to do in order to work with the WDBC dataset is to read it in from the provided .csv data file:

In [ ]:
data = pd.read_csv("wdbc.csv", header=None)
data.head()

Once the data is converted into a numpy array, construct the data matrix X and the associated class vector y (target variable). 

In [ ]:
# Convert to numpy array
dataArray = np.array(data)

# Cast the input data into type float
X = dataArray[:,2:].astype(float)
y = dataArray[:,1]

It is always a good practice to check the dimensionality of the input data as soon as imported

In [ ]:
print "X dimensions:", X.shape
print "y dimensions:", y.shape

In our current dataset, the y values (labels) are categorical (they can only take one of a discrete set of values) and have a non-numeric representation ("M" or "B"). This is problematic for scikit-learn algorithms, since they assume numerical values, so we need to map the text categories to numerical representations.

In [ ]:
le = preprocessing.LabelEncoder()
le.fit(y)
yTransformed = le.transform(y)
# print yTransformed

An important thing to understand before applying any classification algorithms is how the output labels
are distributed. Are they evenly distributed? Imbalances in distribution of labels can often lead to poor classification results for the minority class even if the classification results for the majority class
are very good.

In [ ]:
yFreq = scipy.stats.itemfreq(y)
print yFreq

It is usually advisable to scare your data prior to fitting a classification model. The main advantage of scaling is to avoid attributes in greater numeric ranges dominating those in smaller numeric ranges. For the purposes of this case study, we are applying auto-scaling on the whole dataset (Auto-scaling: mean-centering performed per column; subsequently, scaling is applied by dividing the centered columns by their standard deviation). 

In [ ]:
Xauto = preprocessing.StandardScaler().fit_transform(X)

You can visualise the relationship between two variables (radius and perimeter of the tumour) using a simple scatter plot. To illustrate, let’s plot the first two variables against each other:

In [ ]:
plt.scatter(x = Xauto[yTransformed==0,0], y = Xauto[yTransformed==0,1], color = 'b', label='Benign')
plt.scatter(x = Xauto[yTransformed==1,0], y = Xauto[yTransformed==1,1], color = 'r', label='Malignant')
plt.xlabel("radius")
plt.ylabel("perimeter")
plt.legend()
plt.show()

In [ ]:
plt.hist(Xauto[yTransformed==0,0], color='b', alpha=.5, label='Benign', bins=15)
plt.hist(Xauto[yTransformed==1,0], color='r', alpha=.5, label='Malignant', bins=15)
plt.xlabel("radius")
plt.ylabel("occurrences")
plt.legend()
plt.show()

## Training and testing a classifier

Training and testing a classification model on the same dataset is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data (poor generalisation). <br/> 

To use different datasets for training and testing, we need to split the breast cancer dataset into two disjoint sets: train and test (**Holdout method**). <br/> 

In [ ]:
XTrain, XTest, yTrain, yTest = cv.train_test_split(Xauto, yTransformed, test_size= 0.3, random_state=5)

XTrain and yTrain are the two arrays you use to train your model. XTest and yTest are the two arrays that you use to evaluate your model. By default, scikit-learn splits the data so that 25% of it is used for testing, but you can also specify the proportion of data you want to use for training and testing (in this case, 30% is used for testing).

You can check the sizes of the different training and test sets by using the shape attribute:

In [ ]:
print "XTrain dimensions:", XTrain.shape
print "yTrain dimensions:", yTrain.shape
print "XTest dimensions:",  XTest.shape
print "yTest dimensions:",  yTest.shape

## K Nearest Neighbors (KNN)

To build KNN models using scikit-learn, you will be using the **KNeighborsClassifier** function, which allows you to set the value of K using the *n_neighbors* parameter. The optimal choice of the value K is highly data-dependent: in general a larger K suppresses the effects of noise, but makes the classification boundaries less distinct. <br/>

### Uniform weights

We are going to start by trying two predefined random values of K and compare their performance:

In [ ]:
knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(XTrain, yTrain)
yPredK3 = knn3.predict(XTest)

print metrics.classification_report(yTest, yPredK3)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, yPredK3), 2)

In [ ]:
knn10 = KNeighborsClassifier(n_neighbors=10)
knn10.fit(XTrain, yTrain)
yPredK10 = knn10.predict(XTest)

print metrics.classification_report(yTest, yPredK10)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, yPredK10), 2)

### Distance weights

Under some circumstances, it is better to give more importance ("weight" in computing terms) to nearer neighbours. When weights = "distance", weights are assigned to the training data points in a way that is proportional to the inverse of the distance from the query point. In other words, nearer neighbours contribute more to the fit. <br/>

What if we use weights based on distance? Does it improve the overall performance? 

In [ ]:
knnW3 = KNeighborsClassifier(n_neighbors=3, weights='distance')
knnW3.fit(XTrain, yTrain)
predictedW3 = knnW3.predict(XTest)

print metrics.classification_report(yTest, predictedW3)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, predictedW3), 2)

### Comparison

In [ ]:
def plotvector(XTrain, yTrain, XTest, yTest, weights, upperLim = 310):
    results = []
    for n in range(1, upperLim, 4):
        clf = KNeighborsClassifier(n_neighbors = n, weights = weights)
        clf = clf.fit(XTrain, yTrain)
        preds = clf.predict(XTest)
        accuracy = clf.score(XTest, yTest)
        results.append([n, accuracy])
    results = np.array(results)
    return(results)

def plotaccuracy(XTrain, yTrain, XTest, yTest, upperLim):
    pltvector1 = plotvector(XTrain, yTrain, XTest, yTest, weights = "uniform",  upperLim=upperLim)
    pltvector2 = plotvector(XTrain, yTrain, XTest, yTest, weights = "distance", upperLim=upperLim)
    line1 = plt.plot(pltvector1[:,0], pltvector1[:,1], label = "uniform",  color='blue')
    line2 = plt.plot(pltvector2[:,0], pltvector2[:,1], label = "distance", color='red')
    plt.legend(loc=3)
    plt.ylim(0.85, 1)
    plt.title('Accuracy with Increasing K')
    plt.xlabel('Number of K nearest neighbors')
    plt.ylabel('Classifiction Accuracy')
    plt.grid()

This function plots the accuracies of applying the KNN algorithms with different k values for both uniform and distance weights up to a k value of 100.

In [ ]:
plt.figure(figsize=(7, 6))
plotaccuracy(XTrain, yTrain, XTest, yTest, 200)

### Tuning KNN parameters

The sklearn library has a grid search function, GridSearchCV, that allows us to search for the optimum
combination of parameters by evaluating models trained with a particular algorithm with all
parameter combinations. You can use this function to search for a parametisation of the KNN algorithm
that gives a more optimal model.

In [ ]:
# We want to use odd numbers of k to avoid ties

n_neighbors = np.arange(1, 50, 2)
weights     = ['uniform','distance']

# GridSearchCV accepts parameter values  only as a dictionary

parameters = [{'n_neighbors': n_neighbors, 'weights': weights}]
grid = GridSearchCV(KNeighborsClassifier(), parameters, cv= 10)
grid.fit(XTrain, yTrain)

print "The best parameters are: n_neighbors=", grid.best_params_['n_neighbors'],"and weight=",grid.best_params_['weights']


In [ ]:
score_dict = grid.grid_scores_
scores = [x[1] for x in score_dict]
scores = np.array(scores).reshape(len(n_neighbors), len(weights))
scores = np.transpose(scores)

# Make a heatmap with the performance
plt.figure(figsize=(12, 6))
plt.imshow(scores, interpolation='nearest', origin='higher', cmap=plt.cm.get_cmap('jet_r'))

plt.colorbar()
plt.xticks(np.arange(len(n_neighbors)), n_neighbors)
plt.yticks(np.arange(len(weights)), weights)
plt.xlabel('Number of K nearest neighbors')
plt.ylabel('Weights')
plt.show()

When evaluating the resulting model it is important to do it on held-out samples that were not seen during the grid search process (XTest). <br/> So, we are testing our independent XTest dataset using the optimised model:

In [ ]:
knn = KNeighborsClassifier(n_neighbors=grid.best_params_['n_neighbors'], weights = grid.best_params_['weights'])
knn.fit(XTrain, yTrain)
yPredKnn = knn3.predict(XTest)

print metrics.classification_report(yTest, yPredKnn)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, yPredKnn), 2)

In [ ]:
from sklearn.grid_search import RandomizedSearchCV
from scipy.stats.distributions import randint
param_dist = {'n_neighbors': randint(1,200)}
random_search = RandomizedSearchCV(KNeighborsClassifier(), param_distributions=param_dist, n_iter=20)
random_search.fit(XTrain, yTrain)

print "The best parameters are: n_neighbors=", random_search.best_params_['n_neighbors']

neig = [score_tuple[0]['n_neighbors'] for score_tuple in random_search.grid_scores_]
res = [score_tuple[1] for score_tuple in random_search.grid_scores_]
plt.scatter(neig, res)
plt.xlabel('n_neighbors')
plt.show()

## Random Forest (RF)

Random forests aggregates a group of decision trees into an ensembles. It adds randomness in 2 ways, one is by sampling with replacement(bootstrap sampling) from the training data and then fitting a tree for each of these samples. Then splitting on a feature in the decision tree, random forest considers random subset of variables to split on. <br/>

One of the most important tuning parameters in building a random forest is the number of trees to construct.

In [ ]:
clf = RandomForestClassifier(n_jobs=2)
clf.fit(XTrain, yTrain)
predRF = clf.predict(XTest)

print metrics.classification_report(yTest, predRF)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, predRF),2)

### Tuning RF parameters

The main parameters to adjust when using these methods is n_estimators and max_features. The former is the number of trees in the forest. The larger the better, but also the longer it will take to compute. In addition, note that results will stop getting significantly better beyond a critical number of trees. The latter is the size of the random subsets of features to consider when splitting a node. The lower the greater the reduction of variance, but also the greater the increase in bias.

## Support Vector Machines (SVMs)

SVMs attempt to build a decision boundary that accurately separates the samples of different classes by *maximizing* the margin between them.

### Linear SVMs

The parameter C, common to all SVM kernels, trades off misclassification of training examples against simplicity of the decision surface. A low C makes the decision surface smooth, while a high C aims at classifying all training examples correctly. 

In this example, we will use linear SVMs with the default value for C

In [ ]:
# Default C parameter
linearSVM = SVC(kernel='linear')
linearSVM.fit(XTrain, yTrain)
yPredLinear = linearSVM.predict(XTest)

print metrics.classification_report(yTest, yPredLinear)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, yPredLinear),2)

### Non-linear SVMs

In addition to C, which is common for all types of SVM, the gamma parameter defines the nonlinearity of the bounaries. The larger the gamma is, the more nonlinear the boundaries surrounding individual samples. <br/> In this example, we will use non-linear SVMs with the default values for C and gamma

In [ ]:
# Default C and gamma parameters
rbfSVM = SVC(kernel='rbf', C=1.0, gamma=0.0)
rbfSVM.fit(XTrain, yTrain)
yPredRBF = rbfSVM.predict(XTest)

print metrics.classification_report(yTest, yPredRBF)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, yPredRBF),2)

### Hyperparameter Tuning

Proper choice of C and gamma is critical for the performance of SVMs. Optimisation (tuning) of the hyperparameters can be achieved by applying a coarse tuning (often followed by a finer-tuning in the "neighborhood" of good parameters)

In [ ]:
# Range for gamma and Cost hyperparameters
g_range = 2. ** np.arange(-15, 5, step=2)
C_range = 2. ** np.arange(-5, 15, step=2)

grid = [{'gamma': g_range, 'C': C_range}]

gridcv = GridSearchCV(SVC(), param_grid=grid, cv= cv.KFold(n=XTrain.shape[0], n_folds=5))
gridcv.fit(XTrain, yTrain)

bestG = np.log2(gridcv.best_params_['gamma']);
bestC = np.log2(gridcv.best_params_['C']);

print "The best parameters are: gamma=", bestG, " and Cost=", bestC

Plot the results of the grid search using a heatmap

In [ ]:
# plot the scores of the grid
# grid_scores_ contains parameter settings and scores
score_dict = gridcv.grid_scores_

# We extract just the scores
scores = [x[1] for x in score_dict]
scores = np.array(scores).reshape(len(C_range), len(g_range))

# Make a heatmap with the performance
plt.figure(figsize=(10, 6))
plt.subplots_adjust(left=0.15, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest', origin='higher', cmap=plt.cm.get_cmap('jet_r'))
plt.xlabel('gamma (log2)')
plt.ylabel('Cost (log2)')
plt.colorbar()
plt.xticks(np.arange(len(g_range)), np.log2(g_range))
plt.yticks(np.arange(len(C_range)), np.log2(C_range))
plt.show()

Finally, testing with the optimised model (best hyperparameters):

In [ ]:
rbfSVM = SVC(kernel='rbf', C=gridcv.best_params_['C'], gamma=gridcv.best_params_['gamma'])
rbfSVM.fit(XTrain, yTrain)

predictions = rbfSVM.predict(XTest) 

print metrics.classification_report(yTest, predictions)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, predictions),2)

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
l_regression = LogisticRegression()
l_regression.fit(XTrain, yTrain)
net_prediction = l_regression.predict(XTest)

print metrics.classification_report(yTest, net_prediction)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, net_prediction),2)

In [ ]:
# Range for gamma and Cost hyperparameters
pen = ['l1','l2']
C_range = 2. ** np.arange(-5, 15, step=2)

grid = [{'C': C_range, 'penalty': pen}]

gridcv = GridSearchCV(LogisticRegression(), param_grid=grid, cv= cv.KFold(n=XTrain.shape[0], n_folds=5))
gridcv.fit(XTrain, yTrain)

best_c = gridcv.best_params_['C']
best_penalty = gridcv.best_params_['penalty']

print "The best parameters are: cost=", best_c, " and penalty=", best_penalty

## Neural Networks

In [ ]:
from multilayer_perceptron import multilayer_perceptron

In [ ]:
nnet = multilayer_perceptron.MultilayerPerceptronClassifier(activation='logistic',
                                                            hidden_layer_sizes=(2), learning_rate_init=.5)
nnet.fit(XTrain, yTrain)
net_prediction = nnet.predict(XTest)

print metrics.classification_report(yTest, net_prediction)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, net_prediction),2)

In [ ]:
# Range for gamma and Cost hyperparameters
layer_size_range = [(3,2),(10,10),(2,2,2),10,5] # different networks shapes
learning_rate_range = np.linspace(.1,1,3)

grid = [{'hidden_layer_sizes': layer_size_range, 'learning_rate_init': learning_rate_range}]

gridcv = GridSearchCV(multilayer_perceptron.MultilayerPerceptronClassifier(), 
                      param_grid=grid, cv= cv.KFold(n=XTrain.shape[0], n_folds=5))
gridcv.fit(XTrain, yTrain)

best_size = gridcv.best_params_['hidden_layer_sizes']
best_best_lr = gridcv.best_params_['learning_rate_init']

print "The best parameters are: hidden_layer_sizes=", best_size, " and learning_rate_init=", best_best_lr

In [ ]:
nnet = multilayer_perceptron.MultilayerPerceptronClassifier(hidden_layer_sizes=(2, 2, 2), learning_rate_init=.1)
nnet.fit(XTrain, yTrain)
net_prediction = nnet.predict(XTest)

print metrics.classification_report(yTest, net_prediction)
print "Overall Accuracy:", round(metrics.accuracy_score(yTest, net_prediction),2)

In [ ]:
cv.cross_val_score